63f78014766fd30436c18a79_Hyperspace - navbar logo.png

#  Academic Papers Hybrid Search with Hyperspace
This notebook demonstrates the use of Hyperspace engine for a hybrid search for academic papers matching - a combination of vector search and keyword matching.
We will build a simple filtering function, which filters papers of the same category, gives bias to paper by same submitter an negative bias for papers without given license. We first select a paper as input for the query

# The Datset
The dataset is taken from [kaggle]( https://www.kaggle.com/datasets/Cornell-University/arxiv) and includes a list of academic papers from arXiv, and their metadata.
We will use the combination of an embedded vector data and metadata, to create a hybrid search.


## The Metadata Fields
The metadata includes the following fields:
1. **id** [float] - paper unique id
2. **title** [string] - paper title
3. **submitter** [string] - name of person who submitted the paper
4. **categories** [list[string]] - list of categories which include the paper
5. **label** [list[string]] - labels aplied to paper
6. **license** [string] - license type
7. **update_date_ts** [integer] - update time in unix format


# Setting up the Hyperspace environment
Working with Hyperspace requires the followin steps

1. Install the client API
2. Create data config file
3. Connect to a server
4. Create collection
5. Ingest data
6. Run query

## 1. Install the client API
Hyperspace API can be installed directly from git, using the following command

In [ ]:
pip install git+https://github.com/hyper-space-io/hyperspace-py

  Cloning https://github.com/hyper-space-io/hyperspace-py to /tmp/pip-req-build-yue8jaqq
  Running command git clone --filter=blob:none --quiet https://github.com/hyper-space-io/hyperspace-py /tmp/pip-req-build-yue8jaqq
  Resolved https://github.com/hyper-space-io/hyperspace-py to commit 43ea7a0e58f97c21460cf64e860c3d8f7c29d20d
  Preparing metadata (setup.py) ... done
  Created wheel for hyperspace: filename=hyperspace-1.0.0-py3-none-any.whl size=32263 sha256=0fb4b7df02cd1eaa29f173bbe9a7eab0f737c9b97d846b1eef7e9a85d1b0b2c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-l7b5_pwq/wheels/c4/96/59/f4b91d653fdbfc819e48a7dacbea1c9f3de59a1bc113aa840d
Successfully built hyperspace


#2. Connect to a server

Once the Hyperspace API is installed, the database can be accessed by creating a local instance of the Hyperspace client. This step requires host address, username and password

In [ ]:
import hyperspace

host = 'https://search-master-demo.development.hyper-space.xyz'

hyperspace_client = hyperspace.HyperspaceClientApi(host=host,
                                                   username=username, password=password)


We check the status before proceeding

In [ ]:
cluster_status = hyperspace_client.cluster_status()
display(cluster_status)

[{'FPGA memory usage in GB': '0.0078GB',
  'FPGA memory usage in percentage': '0.0078%',
  'Hostname': 'hyperspace-demo-0',
  'Number of total vectors': 0},
 {'Number of data nodes': 1}]

#3.   Create a Data Schema File

Similarly to other search databases, Hyper-Space database requires a configuration file that outlines the data schema. Here, we create a config file that corresponds to the fields of the given dataset.

For vector fields, we also provide the index type to be used, and the metric. . Current options for index include "**brute_force**", "**hnsw**", "**ivf**", and "**bin_ivf**" for binary vectors, and "**IP**" (inner product) as a metric for floating point vectors and "**Hamming**" ([hamming distance](https://en.wikipedia.org/wiki/Hamming_distance)) for binary vectors.
Here, we use "brute_force" (exact KNN) with inner product.

In [ ]:
import json

config = {
    "configuration": {
        "id": {
            "type":"float"
        },
        "title": {
            "type":"keyword"
        },
        "submitter": {
            "type":"keyword"
        },
        "categories": {
            "type":"keyword",
            "struct_type":"list"
        },
        "labels": {
            "type":"keyword",
            "struct_type":"list"
        },
        "license": {
            "type":"keyword"
        },
        "update_date": {
            "type":"keyword"
        },
        "update_date_ts": {
            "type":"integer"
        },
        "embedded_abstract": {
            "type": "dense_vector",
            "dim": 384,
            "index_type": "brute_force",
            "metric": "IP"
        }
    }
}

with open('arXiv_config.json', 'w') as f:
    f.write(json.dumps(config, indent=2))



## 4. Create Collection
The Hyerspace engine stroes data in Collections, where each collecction commonly hosts data of similar context, etc. Each search is then perfomed within a collection. We create a collection using the command "**create_collection**(schema_filename, collection_name)".

In [ ]:
collection_name = 'all-MiniLM-L6-v2 ArXiv titles'

# hyperspace_client.delete_collection(collection_name)
hyperspace_client.create_collection('arXiv_config.json', collection_name)
hyperspace_client.cluster_status()

# 5. Ingest data

In the next step we ingest the dataset in batches of 250 documents. This number can be controlled by user, and in particular, can be increased in order improve ingestion time. We add batches of data using the command **add_batch**(batch, collection_name).

In [ ]:
import numpy as np
vecs = np.load('vectors.npy')
metadata_file = open('payloads.jsonl')

In [ ]:
BATCH_SIZE = 500

batch = []
for i, (metadata_row, vec) in enumerate(zip(metadata_file, vecs)):
    row = {key: value for key, value in json.loads(metadata_row).items() if key in config["configuration"].keys()}
    row['categories'] = row['categories'].split()
    row['embedded_abstract'] = np.ndarray.tolist(vec)

    batch.append(hyperspace.Document(str(i), row))

    if i % BATCH_SIZE == 0:
        response = hyperspace_client.add_batch(batch, collection_name)
        batch.clear()
        print(i, response)

hyperspace_client.commit(collection_name)

# 6. Define Logic and Run a Query
We will build a hybrid search query using Hyperspace. In the query,  we will randomly select a paper from the database and search for similar papers. The query object weights under the "boost" fields, allow to contorl the relative weights of the classic search and vector search scores. We will first select the input document, and search for similar papers in the next step.

In [ ]:
input_document = hyperspace_client.get_document(collection_name, 65)
print(input_document['title'], "\n======================================\n", input_document['submitter'], "\n", input_document['categories'])

Hall field induced magnetoresistance oscillations of a two-dimensional
  electron system 
 Alejandro Kunold 
 ['cond-mat.mes-hall']


In [ ]:
from pprint import pprint
response = hyperspace_client.set_function('score_func.py', collection_name=collection_name, function_name='similarity_sf')

query_with_knn = {
    'vector_Content': input_document,
    'boost': {
        'query': 0.4,
        'embedded_abstract': 0.2
    }
}

results = hyperspace_client.search_data(query_with_knn,
                                        size=5,
                                        function_name='similarity_sf',
                                        collection_name=collection_name)

for i, result in enumerate(results['similarity']):
  vector_api_response = hyperspace_client.find_document(vector_id=result['vector_id'], collection_name=collection_name)
  print(i + 1, "vector id", result['vector_id'],  ":", vector_api_response['title'], ",", vector_api_response['submitter'], ",", vector_api_response['categories'])
  print("\n")


1 . vector id 65 : Hall field induced magnetoresistance oscillations of a two-dimensional
  electron system , Alejandro Kunold , ['cond-mat.mes-hall']


2 . vector id 366812 : Quantum and classical dissipation of charged particles , Alejandro Kunold , ['quant-ph', 'cond-mat.mes-hall']


3 . vector id 117319 : Non linear transport theory for negative-differential resistance states
  of two dimensional electron systems in strong magnetic fields , Alejandro Kunold , ['cond-mat.mes-hall']


4 . vector id 242529 : Symmetry breaking as the origin of zero-differential resistance states
  of a 2DEG in strong magnetic fields , Alejandro Kunold , ['cond-mat.mes-hall', 'cond-mat.other']


5 . vector id 178121 : Impact of heavy hole-light hole coupling on optical selection rules in
  GaAs quantum dots , Alejandro Kunold , ['cond-mat.mes-hall']




The returned documents have similar submitter name, as expected from the metadata filtering.

This notebook gave a simple example of the use of the Hyperspace engine for hybrid search. Hyperspace can support complicated use cases with large databases, in extremley low latency.